In [5]:
import pandas as pd
import numpy as np
import sqlite3
import os

In [7]:
current_directory = os.getcwd()
print(current_directory)

C:\Users\matth\OneDrive\Desktop\Assignments\Lab_Assignment


In [9]:
con = sqlite3.connect(r'./data/python.db')

school_data = pd.read_sql_query("SELECT * FROM school", con)
school_data
student_data = pd.read_sql_query("SELECT * FROM student", con)
student_data

,id,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,4,Bonnie Ray,F,9th,Huang High School,97,84
...,...,...,...,...,...,...,...,...
39165,39165,39165,Donna Howard,F,12th,Thomas High School,99,90
39166,39166,39166,Dawn Bell,F,10th,Thomas High School,95,70
39167,39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84
39168,39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90


In [11]:
# join the two tables into a single dataframe
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,id_x,Student ID,student_name,gender,grade,school_name,reading_score,math_score,id_y,School ID,type,size,budget
0,0,0,Paul Bradley,M,9th,Huang High School,66,79,0,0,District,2917,1910635
1,1,1,Victor Smith,M,12th,Huang High School,94,61,0,0,District,2917,1910635
2,2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,0,District,2917,1910635
3,3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,0,District,2917,1910635
4,4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,0,District,2917,1910635


In [13]:
# Total number of schools
total_schools = school_data["school_name"].nunique()

# Total number of students
total_students = student_data["Student ID"].count()

# Total budget (sum of all school budgets)
total_budget = school_data["budget"].sum()

# Average math score
average_math_score = student_data["math_score"].mean()

# Average reading score
average_reading_score = student_data["reading_score"].mean()

# Overall average score
overall_average_score = (average_math_score + average_reading_score) / 2

# Percentage of students passing math (70 or greater)
passing_math_percentage = (student_data["math_score"] >= 70).mean() * 100


In [15]:
# Create a District Summary
district_summary = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "Overall Average Score": [overall_average_score],
    "% Passing Math": [passing_math_percentage]
})

# Display the district summary
district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Overall Average Score,% Passing Math
0,15,39170,24649428,78.985371,81.87784,80.431606,74.980853


In [17]:
# Total number of schools
total_schools = school_data["school_name"].nunique()
print(total_schools)

15


In [19]:
# Total number of students
total_students = student_data["Student ID"].nunique()
print(total_students)

39170


In [21]:
# Total budget
total_budget = school_data["budget"].sum()
print(total_budget)

24649428


In [23]:
# Average math score
average_math_score = student_data["math_score"].mean()
print(average_math_score)

78.98537145774827


In [25]:
# Average reading score
average_reading_score = student_data["reading_score"].mean()
print(average_reading_score)

81.87784018381414


In [27]:
# Overall average score
overall_average_score = (student_data["math_score"].mean() + student_data["reading_score"].mean()) / 2
print(overall_average_score)

80.43160582078121


In [29]:
# Percentage of passing math (70 or greater)
passing_math_percentage = (student_data[student_data["math_score"] >= 70]["Student ID"].count() / total_students) * 100
print(passing_math_percentage)

74.9808526933878


In [31]:
# percentage of passing reading (70 or greater)
passing_reading_percentage = (student_data[student_data["reading_score"] >= 70]["Student ID"].count() / total_students) * 100
print(passing_reading_percentage)

85.80546336482001


In [51]:
# Group by school name
school_grouped = school_data_complete.groupby("school_name")

# Calculate total students per school
per_school_counts = school_grouped["Student ID"].count()

# Calculate total school budget and per student budget
per_school_budget = school_grouped["budget"].mean()
per_student_budget = per_school_budget / per_school_counts

# Calculate average math and reading scores
per_school_math = school_grouped["math_score"].mean()
per_school_reading = school_grouped["reading_score"].mean()

# Calculate % passing math
passing_math = school_data_complete[school_data_complete["math_score"] >= 70]
per_school_passing_math = passing_math.groupby("school_name")["Student ID"].count() / per_school_counts * 100

# Calculate % passing reading
passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70]
per_school_passing_reading = passing_reading.groupby("school_name")["Student ID"].count() / per_school_counts * 100


# Calculate % overall passing as the average of math and reading passing rates
per_school_overall_passing = (per_school_passing_math + per_school_passing_reading) / 2


# Build school summary DataFrame
school_summary = pd.DataFrame({
    "School Type": school_grouped["type"].first(),
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_student_budget,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing": per_school_overall_passing
})

# Show the first few rows
top_5_schools = school_summary.sort_values(by="% Overall Passing", ascending=False).head(5)
top_5_schools


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,1043130.0,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574.0,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


In [53]:
# Find the total counts of math result
math_total_counts = school_data_complete["math_score"].count()
math_total_counts

39170

In [55]:
# Find the counts for math result in each school that pass 70 or higher
passing_math_counts = school_data_complete[school_data_complete["math_score"] >= 70].groupby("school_name")["math_score"].count()
passing_math_counts

school_name
Bailey High School       3318
Cabrera High School      1749
Figueroa High School     1946
Ford High School         1871
Griffin High School      1371
Hernandez High School    3094
Holden High School        395
Huang High School        1916
Johnson High School      3145
Pena High School          910
Rodriguez High School    2654
Shelton High School      1653
Thomas High School       1525
Wilson High School       2143
Wright High School       1680
Name: math_score, dtype: int64

In [57]:
# Calculate the math passing rate
per_school_math_count = school_data_complete.groupby("school_name")["math_score"].count()
math_pass_rate = (passing_math_counts / per_school_math_count) * 100
math_pass_rate

school_name
Bailey High School       66.680064
Cabrera High School      94.133477
Figueroa High School     65.988471
Ford High School         68.309602
Griffin High School      93.392371
Hernandez High School    66.752967
Holden High School       92.505855
Huang High School        65.683922
Johnson High School      66.057551
Pena High School         94.594595
Rodriguez High School    66.366592
Shelton High School      93.867121
Thomas High School       93.272171
Wilson High School       93.867718
Wright High School       93.333333
Name: math_score, dtype: float64

In [43]:
# Find the total counts of read result
read_total_counts = school_data_complete["reading_score"].count()
read_total_counts

39170

In [45]:
# Find the counts for read result in each school that pass 70 or higher
passing_read_counts = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("school_name")["reading_score"].count()
passing_read_counts

school_name
Bailey High School       4077
Cabrera High School      1803
Figueroa High School     2381
Ford High School         2172
Griffin High School      1426
Hernandez High School    3748
Holden High School        411
Huang High School        2372
Johnson High School      3867
Pena High School          923
Rodriguez High School    3208
Shelton High School      1688
Thomas High School       1591
Wilson High School       2204
Wright High School       1739
Name: reading_score, dtype: int64

In [47]:
# Calculate the read passing rate
per_school_read_count = school_data_complete.groupby("school_name")["reading_score"].count()
read_pass_rate = (passing_read_counts / per_school_read_count) * 100
read_pass_rate

school_name
Bailey High School       81.933280
Cabrera High School      97.039828
Figueroa High School     80.739234
Ford High School         79.299014
Griffin High School      97.138965
Hernandez High School    80.862999
Holden High School       96.252927
Huang High School        81.316421
Johnson High School      81.222432
Pena High School         95.945946
Rodriguez High School    80.220055
Shelton High School      95.854628
Thomas High School       97.308869
Wilson High School       96.539641
Wright High School       96.611111
Name: reading_score, dtype: float64

In [49]:
# Calculate the overall passing rate (average of the math and reading passing rate)
overall_pass_rate = (math_pass_rate + read_pass_rate) / 2
overall_pass_rate


school_name
Bailey High School       74.306672
Cabrera High School      95.586652
Figueroa High School     73.363852
Ford High School         73.804308
Griffin High School      95.265668
Hernandez High School    73.807983
Holden High School       94.379391
Huang High School        73.500171
Johnson High School      73.639992
Pena High School         95.270270
Rodriguez High School    73.293323
Shelton High School      94.860875
Thomas High School       95.290520
Wilson High School       95.203679
Wright High School       94.972222
dtype: float64

In [61]:
# Sort and display the worst five schools in overall passing rate
worst_5_schools = overall_pass_rate.sort_values(ascending=True).head(5)
worst_5_schools


school_name
Rodriguez High School    73.293323
Figueroa High School     73.363852
Huang High School        73.500171
Johnson High School      73.639992
Ford High School         73.804308
dtype: float64

In [63]:
# Create table that lists the average math score for each school of each grade level
math_by_grade = school_data_complete.pivot_table(index="school_name", columns="grade", values="math_score")
math_by_grade

grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,76.996772,77.515588,76.492218,77.083676
Cabrera High School,83.154506,82.765560,83.277487,83.094697
Figueroa High School,76.539974,76.884344,77.151369,76.403037
Ford High School,77.672316,76.918058,76.179963,77.361345
Griffin High School,84.229064,83.842105,83.356164,82.044010
Hernandez High School,77.337408,77.136029,77.186567,77.438495
Holden High School,83.429825,85.000000,82.855422,83.787402
Huang High School,75.908735,76.446602,77.225641,77.027251
Johnson High School,76.691117,77.491653,76.863248,77.187857


In [65]:
# Create table that lists the average reading score for each school of each grade level
reading_by_grade = school_data_complete.pivot_table(index="school_name", columns="grade", values="reading_score")
reading_by_grade


grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,80.907183,80.945643,80.912451,81.303155
Cabrera High School,84.253219,83.788382,84.287958,83.676136
Figueroa High School,81.408912,80.640339,81.384863,81.198598
Ford High School,81.262712,80.403642,80.662338,80.632653
Griffin High School,83.706897,84.288089,84.013699,83.369193
Hernandez High School,80.660147,81.396140,80.857143,80.866860
Holden High School,83.324561,83.815534,84.698795,83.677165
Huang High School,81.512386,81.417476,80.305983,81.290284
Johnson High School,80.773431,80.616027,81.227564,81.260714


In [69]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [71]:
# Create a new column to show budget per student in each row
school_data_complete["Per Student Budget"] = school_data_complete["budget"] / school_data_complete["size"]

In [73]:
# Create a new column to define the spending ranges per student
school_data_complete["Spending Ranges (Per Student)"] = pd.cut(school_data_complete["Per Student Budget"], spending_bins, labels=group_names)

In [77]:
# Calculate the average math score within each spending range
spending_math_score = school_data_complete.groupby("Spending Ranges (Per Student)", observed=True)["math_score"].mean()

In [83]:
# Calculate the percentage passing rate for math in each spending range
spending_math_passing = school_data_complete[school_data_complete["math_score"] >= 70] \
    .groupby("Spending Ranges (Per Student)", observed=True)["math_score"].count() / \
    school_data_complete.groupby("Spending Ranges (Per Student)", observed=True)["math_score"].count() * 100

# Calculate the percentage passing rate for reading in each spending range
spending_read_passing = school_data_complete[school_data_complete["reading_score"] >= 70] \
    .groupby("Spending Ranges (Per Student)", observed=True)["reading_score"].count() / \
    school_data_complete.groupby("Spending Ranges (Per Student)", observed=True)["reading_score"].count() * 100

# Calculate the percentage overall passing rate in each spending range
spending_overall_passing = (spending_math_passing + spending_read_passing) / 2

# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Create a new column for the bin groups
school_data_complete["School Size"] = pd.cut(school_data_complete["size"], size_bins, labels=group_names)
school_data_complete["School Size"].value_counts()

School Size
Large (2000-5000)     29259
Medium (1000-2000)     8522
Small (<1000)          1389
Name: count, dtype: int64

In [85]:
# math_pass_size
math_pass_size = school_data_complete[school_data_complete["math_score"] >= 70] \
    .groupby("School Size", observed=True)["Student ID"].count() / \
    school_data_complete.groupby("School Size", observed=True)["Student ID"].count() * 100
math_pass_size

School Size
Small (<1000)         93.952484
Medium (1000-2000)    93.616522
Large (2000-5000)     68.652380
Name: Student ID, dtype: float64

In [87]:
# read_pass_size
read_pass_size = school_data_complete[school_data_complete["reading_score"] >= 70] \
    .groupby("School Size", observed=True)["Student ID"].count() / \
    school_data_complete.groupby("School Size", observed=True)["Student ID"].count() * 100
read_pass_size

School Size
Small (<1000)         96.040317
Medium (1000-2000)    96.773058
Large (2000-5000)     82.125158
Name: Student ID, dtype: float64

In [89]:
# Calculate the overall passing rate for different school size
overall_pass_size = (math_pass_size + read_pass_size) / 2
overall_pass_size

School Size
Small (<1000)         94.996400
Medium (1000-2000)    95.194790
Large (2000-5000)     75.388769
Name: Student ID, dtype: float64

In [91]:
# math_pass_type
math_pass_type = school_data_complete[school_data_complete["math_score"] >= 70] \
    .groupby("type", observed=True)["Student ID"].count() / \
    school_data_complete.groupby("type", observed=True)["Student ID"].count() * 100
math_pass_type

type
Charter     93.701821
District    66.518387
Name: Student ID, dtype: float64

In [93]:
# read_pass_type
read_pass_type = school_data_complete[school_data_complete["reading_score"] >= 70] \
    .groupby("type", observed=True)["Student ID"].count() / \
    school_data_complete.groupby("type", observed=True)["Student ID"].count() * 100
read_pass_type

type
Charter     96.645891
District    80.905249
Name: Student ID, dtype: float64

In [95]:
# overall_pass_type
overall_pass_type = (math_pass_type + read_pass_type) / 2
overall_pass_type

type
Charter     95.173856
District    73.711818
Name: Student ID, dtype: float64